In [ ]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('https://s3.eu-west-1.amazonaws.com/course.oc-static.com/courses/7410486/operations.csv')
data

,date_operation,libelle,montant,solde_avt_ope,categ
0,2023-03-31,DON XX XX XX XX XX XX XX,-1.44,1515.25,AUTRE
1,2023-04-03,CARTE XX XX RAPT XX,-24.00,1513.81,TRANSPORT
2,2023-04-03,CARTE XX XX RAPT XX,-73.00,1489.81,TRANSPORT
3,2023-04-03,VIREMENT XX XX XX XX XX XX XX XX XX XX XX XX,676.00,1416.81,AUTRE
4,2023-04-03,VIREMENT XX XX XX XX XX XX,4.80,2092.81,AUTRE
...,...,...,...,...,...
304,2023-10-05,CARTE XX XX XX XX XX XX,-10.64,2440.94,AUTRE
305,2023-10-05,CARTE XX XX XX XX,-4.80,2430.30,AUTRE
306,2023-10-06,FORFAIT COMPTE SUPERBANK XX XX XX XX,-1.92,2425.50,COTISATION BANCAIRE
307,2023-10-06,CARTE XX XX CHEZ LUC XX,-10.00,2423.58,RESTAURANT


In [6]:
data.shape

(309, 5)

### Nettoyage des données

#### 1. Erreur de Type

In [7]:
data.dtypes

date_operation     object
libelle            object
montant           float64
solde_avt_ope     float64
categ              object
dtype: object

On remarque que seul le type de la variable date_operation semble ne pas avoir le bon type.
Utilisons la méthode `to_datetime` de pandas pour corriger son type

In [8]:
data['date_operation'] = pd.to_datetime(data['date_operation'])

#### 2- Valeurs manquantes

In [9]:
data.isnull().sum()

date_operation    0
libelle           0
montant           2
solde_avt_ope     0
categ             1
dtype: int64

Nous avons 3 valeurs manquantes dans la variable montant ( 2 valeurs manquantes) et categ (une valeur manquante)

In [12]:
nb_na = data.isnull().sum()
nb_na[nb_na > 0]

montant    2
categ      1
dtype: int64

Affichons les données sur la variable montant

In [15]:
data.loc[data['montant'].isnull(), :]

,date_operation,libelle,montant,solde_avt_ope,categ
107,2023-06-12,CARTE XX XX LES ANCIENS ROBINSON XX,NaN,4667.19,COURSES
269,2023-09-11,CARTE XX XX XX XX,NaN,3401.93,AUTRE


les transactions étant organisées par ordre chronologique, on peut à partir du solde de l'opération suivante déduire le montant qui a été débité pour chacune des ces opérations.

Le montant manquant correspond donc au solde de l'opération suivante, moins le solde de l'opération concernée. Voilà comment faire cela pour nos lignes concernées :

In [23]:
data_nan = data.loc[data['montant'].isnull(), :]

for index in data_nan.index:
  data.loc[index, 'montant'] = data.loc[index+1, 'solde_avt_ope'] - data.loc[index, 'solde_avt_ope']

In [24]:
data.loc[data['categ'].isnull(), :]

,date_operation,libelle,montant,solde_avt_ope,categ
156,2023-07-06,PRELEVEMENT XX TELEPHONE XX XX,-36.48,3295.68,NaN


la valeur manquante est categ qui devrait contenir FACTURE TELEPHONE d'apres l'analyse de notre dataframe 

In [27]:
data.loc[data['categ'].isnull(), 'categ'] = 'FACTURE TELEPHONE'

#### 3- Les Doublons

Utilisons la mehode .duplicated(keep = False) pour verifier les doublons 

In [29]:
data.loc[data[['date_operation', 'libelle', 'montant', 'solde_avt_ope']].duplicated(keep = False), :]

,date_operation,libelle,montant,solde_avt_ope,categ
43,2023-04-25,CARTE XX XX LES ANCIENS ROBINSON XX,-32.67,3647.67,COURSES
44,2023-04-25,CARTE XX XX LES ANCIENS ROBINSON XX,-32.67,3647.67,COURSES


Les informations d'une personne sont répétes sur la l'index 43 dt 43.
Supprimons une information avec la methode .drop_duplicates(subset=['Les variables], inplace=True, ignore_index=True)

In [30]:
data.drop_duplicates(subset=['date_operation', 'libelle', 'montant', 'solde_avt_ope'], inplace=True, ignore_index=True)

In [31]:
data.describe()

,date_operation,montant,solde_avt_ope
count,308,308.000000,308.000000
mean,2023-07-05 10:59:13.246753280,-45.782013,3395.301071
min,2023-03-31 00:00:00,-15000.000000,1416.810000
25%,2023-05-21 06:00:00,-20.447500,3010.737500
50%,2023-07-05 12:00:00,-9.600000,3452.465000
75%,2023-08-21 00:00:00,-2.715000,3787.232500
max,2023-10-06 00:00:00,1071.600000,4709.310000
std,NaN,872.818105,667.109412
